In [1]:
from multiprocessing.pool import ThreadPool
import h5py
import pandas as pd
import numpy as np
from paiutils import audio

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
data = pd.read_csv('D:\\cv-corpus-7.0-2021-07-21\\validated.tsv', delimiter='\t')
data = data[data['up_votes'] > data['down_votes'] * 2]
len(data), len(data['client_id'].unique())

(1216482, 63458)

In [3]:
print(len(data['sentence'].unique()))
counts = data.groupby('sentence')['path'].count()
data = data[data['sentence'].isin(counts[10 <= counts].index)]
len(data), len(data['client_id'].unique()), len(data['sentence'].unique())

671940


(501224, 33141, 11125)

In [4]:
counts = data.groupby('client_id')['path'].count()
data = data[data['client_id'].isin(counts[10 <= counts].index)]
len(data), len(data['client_id'].unique())

(414383, 9907)

In [5]:
data2 = data[data['client_id'].isin(counts[100 < counts].index)]
data = data.drop(data2[~data2.index.isin(data2.groupby('client_id').sample(100).index)].index)
len(data), len(data['client_id'].unique())

(305091, 9907)

In [6]:
print(len(data['sentence'].unique()))
counts = data.groupby('sentence')['path'].count()
data = data[data['sentence'].isin(counts[10 <= counts].index)]
len(data), len(data['client_id'].unique()), len(data['sentence'].unique())

11125


(294250, 9892, 9577)

In [7]:
data.to_csv('D:\\data.tsv', sep='\t')

In [8]:
sample_rate = 16000

def process(path):
    global sample_rate

    x, sr, at = audio.load(f'D:\\cv-corpus-7.0-2021-07-21\\clips\\{path}', rate=sample_rate)
    path = path.split('.')[0] + '.wav'
    x = audio.vad_trim_sides(x, sr, .03, 3)

    if int(len(x) / 1.333) - sample_rate // 4 > len(x):
        while audio.calc_rms(x[:int(len(x) / 1.333)]) > audio.calc_rms(x[int(len(x) / 1.333):-sample_rate // 4]) * 10:
            x = x[:int(len(x) / 1.166)]
        while audio.calc_rms(x[:int(len(x) * .25)]) * 10 < audio.calc_rms(x[int(len(x) * .25):]):
            x = x[int(len(x) * .142):]

    if audio.calc_duration(x, sr) > 10:
        x = audio.vad_trim_all(x, sr, .03, 1)
        if audio.calc_duration(x, sr) > 10:
            x = audio.set_duration(x, sr, 10)
    audio.save(f'D:\\cv_data\\{path}', x, sr)

    return True

with ThreadPool(50) as p:
    p.map(process, data['path'])